<a href="https://colab.research.google.com/github/Haarish-dev/Monty_Hall_Simulation/blob/main/Univariate_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

pd.set_option("display.max_columns",None)
df = pd.read_csv("BFS2.csv")
df.sort_values(by = ["ReqTime"],inplace=True)
df.reset_index(inplace=True)

In [2]:
df.head(22)

,index,ReqID,Address,Channel,DataSize,R/W,SID,WID,TCP,Const,Text,Atomic,Outstanding,ReqTime,RespTime,RespToRespTime,ReadToRead/WriteToWriteTime,Latency
0,0,53465,c002fac0,2,32,R,62,10,31,0,0,0,0,4772,4809,4772,4772,37
1,1,78793,c0013180,3,32,R,61,1,30,0,0,0,0,9691,9728,9691,9691,37
2,2,79001,c000e400,1,32,R,41,4,20,0,0,0,0,9697,9734,9697,9697,37
3,3,79682,c000d700,3,32,R,37,7,18,0,0,0,0,9714,9734,23,23,20
4,4,81429,c002e180,0,32,R,9,9,4,0,0,0,0,9789,9825,9789,9789,36
5,7,111882,c00143e0,0,32,R,16,3,8,0,0,0,0,15531,15568,5742,5742,37
6,8,111970,c000d380,7,32,R,69,3,34,0,0,0,0,15533,15569,15533,15533,36
7,9,112082,c0011800,6,32,R,6,0,3,0,0,0,0,15536,15572,15536,15536,36
8,11,112130,c000f9c0,6,32,R,79,1,39,0,0,0,0,15537,15576,1,1,39
9,10,112236,c0011020,6,32,R,4,0,2,0,0,0,0,15538,15575,1,1,37


In [3]:
def prep_data(data,no):
  X,y = list(),list()
  for i in range(len(data)):
    end = i+no
    if (end>len(data)-1):
      break
    X_in,y_in = data[i:end],data[end]
    X.append(X_in)
    y.append(y_in)
  
  return array(X),array(y)

In [4]:
df_latency = df.loc[0:20,['Latency']]
lat_seq = df_latency['Latency'].to_numpy()
X,y = prep_data(lat_seq,3)

In [5]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

Vanilla LSTM

In [6]:
yhat_avg=0
for i in range(100):
  model = Sequential()
  model.add(LSTM(50, activation='relu', input_shape=(3,1)))
  model.add(Dense(1))
  model.compile(optimizer='Adam', loss='MSE')
  model.fit(X, y, epochs=200, verbose=0)
  x_input = array([27, 37, 37])
  x_input = x_input.reshape((1, 3, 1))
  yhat = model.predict(x_input, verbose=0)
  yhat_avg += yhat

In [9]:
print(float(yhat_avg/100))

38.61830139160156


Stacked LSTM

In [10]:
yhat_avg=0
for i in range(100):
  model = Sequential()
  model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(3, 1)))
  model.add(LSTM(50, activation='relu'))
  model.add(Dense(1))
  model.compile(optimizer='Adam', loss='mse')
  model.fit(X, y, epochs=200, verbose=0)
  x_input = array([27, 37, 37])
  x_input = x_input.reshape((1, 3, 1))
  yhat = model.predict(x_input, verbose=0)
  yhat_avg += yhat

In [11]:
print(float(yhat_avg)/100)

38.5154931640625


Bidirectional LSTM


In [18]:
from keras.layers import Bidirectional

In [22]:
yhat_avg=0
for i in range(100):
 model = Sequential()
 model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(3, n_features)))
 model.add(Dense(1))
 model.compile(optimizer='adam', loss='mse')
 model.fit(X, y, epochs=200, verbose=0)
 x_input = array([27, 37, 37])
 x_input = x_input.reshape((1, 3, 1))
 yhat = model.predict(x_input, verbose=0)
 yhat_avg +=yhat

In [23]:
print(float(yhat_avg/100))

35.78080749511719
